In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df = pd.read_csv('subAfricaDf.csv')

In [3]:
# get categorical columns in train dataset with missing values and store in missing_cat_cols
list_missing_cat_columns = list((df.select_dtypes(['object']).isna().sum() > 0).index)
list_missing_cat_columns

['Country',
 'State',
 'City',
 'Region_name',
 'AttackType',
 'Target',
 'Summary',
 'Group',
 'Target_type',
 'Weapon_type',
 'sub_weapon_type']

In [4]:
# Fill Categorical columns in data with mode
for col in list_missing_cat_columns:
    df[col] = df[col].fillna('Unknown')

In [5]:
## Fill missing values for numeric columns
# get numeric columns with missing values and store in lst_missing_numeric_col
list_missing_numeric_col = list((df.select_dtypes(np.number).isna().sum() > 0).index)
list_missing_numeric_col

['Year',
 'Month',
 'Day',
 'Region',
 'duration_of_incident',
 'Killed',
 'Wounded',
 'Latitude',
 'Longitude',
 'Suicide',
 'Casualities',
 'Success']

In [6]:
# Fill numeric columns in train dataset with mean
for col in list_missing_numeric_col:
    df[col] = df[col].fillna(df[col].median())

In [7]:
df1 = df.copy()

In [47]:
X= df1.drop(['Success','duration_of_incident','Summary','Latitude','State','Longitude','sub_weapon_type','Year', 'Month', 'Day', 'Casualities','City', 'Region', 'Region_name','Killed', 'Wounded','Target'],
            axis=1)
y= df1['Success']

In [51]:
cat_cols=['Group','Weapon_type','Country','AttackType','Target_type']

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

for col in cat_cols:
    X[col] = le.fit_transform(X[col])

In [52]:
np.bincount (y) 
ros = RandomOverSampler (random_state=2)
X, y = ros.fit_resample (X,y)
np.bincount (y) 

array([16277, 16277], dtype=int64)

In [53]:
RandomForestClassifier(random_state = 1)

RandomForestClassifier(random_state=1)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [55]:
reg_rf = RandomForestClassifier()
reg_rf.fit(X_train, y_train)

RandomForestClassifier()

In [56]:
y_pred = reg_rf.predict(X_test)

In [57]:
reg_rf.score(X_train, y_train)

0.8497484928771647

In [58]:
reg_rf.score(X_test, y_test)

0.8378129319612962

In [59]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 0.16218706803870372
MSE: 0.16218706803870372
RMSE: 0.4027245560413516


In [61]:
metrics.r2_score(y_test, y_pred)

0.3511788613666583

In [62]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [63]:
# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [64]:
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = reg_rf, param_distributions = random_grid,scoring='neg_mean_squared_error',n_iter = 10,cv = 5,random_state = 1, n_jobs =1)

In [65]:
rf_random.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=1, scoring='neg_mean_squared_error')

In [74]:
X.shape

(32554, 6)

In [66]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 15}

In [67]:
prediction = rf_random.predict(X_test)

In [68]:
import pickle
# open a file, where you ant to store the data
file = open('terrorist_rf2.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)
pickle.dump(rf_random, file)

In [69]:
model = open('terrorist_rf2.pkl','rb')
forest = pickle.load(model)

In [70]:
y_prediction = forest.predict(X_test)

In [71]:
metrics.r2_score(y_test, y_prediction)

0.3364329263977187